In [9]:
import pandas as pd

train = pd.read_csv('../train.csv')
test = pd.read_csv('../test.csv')

In [10]:
import h2o
from h2o.automl import H2OAutoML
import h2o
from sklearn.model_selection import train_test_split

# Inicializar el servidor H2O
h2o.init()

train = h2o.H2OFrame(train)
test = h2o.H2OFrame(test)

target = 'nobeyesdad'
features = train.columns
features.remove(target)

aml = H2OAutoML(max_models=50, seed=1, project_name="nobeyesdad_prediction")
aml.train(x=features, y=target, training_frame=train)

# Ver el mejor modelo
best_model = aml.leader
print(best_model)


Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: java version "1.8.0_401"; Java(TM) SE Runtime Environment (build 1.8.0_401-b10); Java HotSpot(TM) 64-Bit Server VM (build 25.401-b10, mixed mode)
  Starting server from /Users/ikeryanez/anaconda3/envs/pfg/lib/python3.10/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/cp/8x9lqtyj66dc8l1337z6dk180000gn/T/tmpccv9a361
  JVM stdout: /var/folders/cp/8x9lqtyj66dc8l1337z6dk180000gn/T/tmpccv9a361/h2o_ikeryanez_started_from_python.out
  JVM stderr: /var/folders/cp/8x9lqtyj66dc8l1337z6dk180000gn/T/tmpccv9a361/h2o_ikeryanez_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,Europe/Madrid
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.1
H2O_cluster_version_age:,1 month and 22 days
H2O_cluster_name:,H2O_from_python_ikeryanez_2kddad
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.541 Gb
H2O_cluster_total_cores:,12
H2O_cluster_allowed_cores:,12
H2O_cluster_status:,"locked, healthy"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%
Model Details
H2OStackedEnsembleEstimator : Stacked Ensemble
Model Key: StackedEnsemble_BestOfFamily_1_AutoML_1_20240505_224405


Model Summary for Stacked Ensemble: 
key                                        value
-----------------------------------------  ----------------
Stacking strategy                          cross_validation
Number of base models (used / total)       6/6
# GBM base models (used / total)           1/1
# XGBoost base models (used / total)       1/1
# GLM base models (used / total)           1/1
# DRF base models (used / total)           2/2
# DeepLearning base models (used / total)  1/1
Metalearner algorithm                      GLM
Metalearner fold assignment scheme         Random


In [11]:
# Predicciones en el conjunto de prueba
predictions = best_model.predict(test)

stackedensemble prediction progress: |

███████████████████████████████████████████| (done) 100%


In [16]:
# Si quieres más detalles o métricas específicas:
performance

ModelMetricsMultinomialGLM: stackedensemble
** Reported on test data. **

MSE: 0.020172123027299333
RMSE: 0.1420285993287948
LogLoss: 0.06344219767848899
Null degrees of freedom: 427
Residual degrees of freedom: 339
Null deviance: 1666.7691666687047
Residual deviance: 54.30652121278656
AUC table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).
AUCPR table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class
Insufficient_Weight    Normal_Weight    Obesity_Type_I    Obesity_Type_II    Obesity_Type_III    Overweight_Level_I    Overweight_Level_II    Error      Rate
---------------------  ---------------  ----------------  -----------------  ------------------  --------------------  ---------------------  ---------  --------
65                     1                0                 0                  0                   0                     0                      0.0151515  1 / 66
2                      54               0                 0                  0                   5                     0                      0.114754   7 / 61
0                      0                72                1                  0                   0                     0                      0.0136986  1 / 73
0                      0                0                 63                 0                   0                     0                      0          0 / 63
0                      0                0                 0                  54                  0                     0                      0          0 / 54
0                      1                0                 0                  0                   49                    1                      0.0392157  2 / 51
0                      0                2                 0                  0                   2                     56                     0.0666667  4 / 60
67                     56               74                64                 54                  56                    57                     0.0350467  15 / 428

Top-7 Hit Ratios: 
k    hit_ratio
---  -----------
1    0.964953
2    1
3    1
4    1
5    1
6    1
7    1

In [17]:
import numpy as np
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Asumimos que H2O ya está iniciado y que las predicciones y los datos reales están listos

# Convertir las predicciones de H2O y los valores reales a formatos de NumPy para usar con sklearn
predicted_labels = predictions['predict'].as_data_frame().values.flatten()
true_labels = test['nobeyesdad'].as_data_frame().values.flatten()

# Calcular la precisión general
accuracy = accuracy_score(true_labels, predicted_labels)
print("Accuracy: ", accuracy)

# Calcular y mostrar la matriz de confusión
conf_matrix = confusion_matrix(true_labels, predicted_labels)
print("Confusion Matrix: \n", conf_matrix)

# Calcular y mostrar el informe de clasificación
class_report = classification_report(true_labels, predicted_labels)
print("Classification Report: \n", class_report)


Accuracy:  0.9649532710280374
Confusion Matrix: 
 [[65  1  0  0  0  0  0]
 [ 2 54  0  0  0  5  0]
 [ 0  0 72  1  0  0  0]
 [ 0  0  0 63  0  0  0]
 [ 0  0  0  0 54  0  0]
 [ 0  1  0  0  0 49  1]
 [ 0  0  2  0  0  2 56]]
Classification Report: 
                      precision    recall  f1-score   support

Insufficient_Weight       0.97      0.98      0.98        66
      Normal_Weight       0.96      0.89      0.92        61
     Obesity_Type_I       0.97      0.99      0.98        73
    Obesity_Type_II       0.98      1.00      0.99        63
   Obesity_Type_III       1.00      1.00      1.00        54
 Overweight_Level_I       0.88      0.96      0.92        51
Overweight_Level_II       0.98      0.93      0.96        60

           accuracy                           0.96       428
          macro avg       0.96      0.96      0.96       428
       weighted avg       0.97      0.96      0.96       428



/Users/ikeryanez/anaconda3/envs/pfg/lib/python3.10/site-packages/h2o/frame.py:1983: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install datatable (for Python 3.9 or lower), or polars and pyarrow (for Python 3.10 or above) and activate it using:

with h2o.utils.threading.local_context(polars_enabled=True, datatable_enabled=True):
    pandas_df = h2o_df.as_data_frame()

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"
/Users/ikeryanez/anaconda3/envs/pfg/lib/python3.10/site-packages/h2o/frame.py:1983: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install datatable (for Python 3.9 or lower), or polars and pyarrow (for Python 3.10 or above) and activate it using:

with h2o.utils.threading.local_context(polars_enabled=True, datatable_enabled=True):
    pandas_df = h2o_df.

In [24]:
model_path = h2o.save_model(model=best_model, path="", force=True)